# Methodology section, preparing the data

## loading in the data

In [48]:
import pandas as pd
import numpy as np
from mat73 import loadmat

### import agouti artis for labelled data linkage

In [5]:
# load in the data from artis
agouti_media = pd.read_csv(
    r'C:\Users\gijst\Documents\Master Data Science\Thesis\artis-20230124120010\media.csv')
agouti_observations = pd.read_csv(
    r'C:\Users\gijst\Documents\Master Data Science\Thesis\artis-20230124120010\observations.csv')


### Now we merge the two dataframes on sequenceID and deploymentID

In [34]:
# remove duplicates from agouti_media (multiple photos of same instance)

agouti_media_nodup = agouti_media.drop_duplicates(subset=['sequenceID'])

In [41]:
agouti = pd.merge(agouti_observations, agouti_media_nodup, on=[
                  'sequenceID', 'deploymentID', 'timestamp'], suffixes=('_obs', '_media'))


In [44]:
agouti = agouti.dropna(axis=1)

In [45]:
agouti.columns

Index(['observationID', 'deploymentID', 'sequenceID', 'timestamp',
       'observationType', 'cameraSetup', 'count', 'classificationMethod',
       'classifiedBy', 'classificationTimestamp', 'mediaID_media',
       'captureMethod', 'filePath', 'fileName', 'fileMediatype'],
      dtype='object')

In [49]:
mat = loadmat(r'C:\Users\gijst\Documents\Master Data Science\Thesis\DeepSqueak-master\Detections\artis_16_audio1_2021-11-11_16-00-00_(0) 2022-12-13  3_45 PM.mat')


ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
